### Easy Generate

In [9]:
# 将 DOT 格式内容解析为嵌套数据结构
import json
import re
def parse_dot_to_target_structure(dot_content, graph_name="example_script"):
    """
    解析 DOT 格式内容为目标嵌套数据结构
    """
    target_structure = {"name": graph_name, "type": "CFG", "blocks": [], "functions": [], "classes": []}

    block_pattern = r'Block_(\d+) \[label="([^"]+)"\];'
    edge_pattern = r'Block_(\d+) -> Block_(\d+);'
    subgraph_pattern = r'subgraph cluster_([^ ]+) {.*?label="([^"]+)";(.*?)\n  }'

    # 解析块（Block）
    blocks = {}
    for match in re.findall(block_pattern, dot_content):
        block_id, block_label = match
        blocks[block_id] = {"id": int(block_id), "label": block_label, "successors": []}

    # 解析边（Edges）
    for match in re.findall(edge_pattern, dot_content):
        src, dest = match
        if src in blocks and dest in blocks:
            blocks[src]["successors"].append(blocks[dest])

    # 将块加入主 CFG
    for block in blocks.values():
        target_structure["blocks"].append(block)

    # 解析子图（Subgraphs）
    subgraphs = re.findall(subgraph_pattern, dot_content, re.DOTALL)
    for cluster_id, label, content in subgraphs:
        subgraph_dict = parse_dot_to_target_structure(content, graph_name=label)
        if cluster_id.startswith("func"):
            target_structure["functions"].append(subgraph_dict)
        elif cluster_id.startswith("class"):
            target_structure["classes"].append(subgraph_dict)

    return target_structure

def extract_dot_from_raw_text(res):
    import re
    
    # 使用正则表达式匹配以```dot开头和```结尾之间的内容
    dot_pattern = r'```dot\n(.*?)```'
    # 如果没有找到```dot，尝试匹配digraph开头的内容
    digraph_pattern = r'(digraph.*?\n})'
    
    # 首先尝试匹配```dot格式
    match = re.search(dot_pattern, res, re.DOTALL)
    if match:
        return match.group(1).strip()
    
    # 如果没找到```dot格式，尝试直接匹配digraph内容
    match = re.search(digraph_pattern, res, re.DOTALL) 
    if match:
        return match.group(1).strip()
        
    return ""  # 如果都没找到则返回空字符串

code_example = """
x = 1

def foo(a):
    if a > 0:
        print("positive")
    else:
        print("not positive")

class Bar:
    def methodA(self, y):
        return y + 123
    def methodB(self, z):
        pass
"""


prompt = """
### Context and Instructions

Generate a Control Flow Graph (CFG) in DOT format based on the given Python code. The output should follow these rules:

1. **Structure**:
   - Use subgraphs to represent the hierarchical structure of the code, such as nested functions or classes.
   - Assign a clear and unique `label` to each subgraph, using the format `cluster_<type>_<name>` (e.g., `cluster_Main`, `cluster_func_<function_name>`).
   - Treat inner functions and classes as complete units within the parent graph.

2. **Node Representation**:
   - Each node represents a basic block or a meaningful control flow element.
   - Use the format: `Block_<ID> [label="#<ID>\\n<code snippet>"];`.
   - Include a concise snippet of the corresponding code in the node's label.

3. **Edge Representation**:
   - Use directed edges to represent the control flow between nodes.
   - If a condition leads to multiple branches (e.g., an `if` statement), include separate edges for `True` and `False` paths.

4. **Node Merging**:
   - Merge consecutive statements without control flow into a single node.
   - If a node only has one child, and that child only has one parent, combine them into a single node.
   - Include the combined code snippets in the node label, separated by `...`.

5. **Simplification**:
   - The goal is to make the graph as concise as possible while preserving the control flow structure.
   - Avoid duplicating nodes unnecessarily and try to reduce visual clutter.

6 **IMPORTANT**:
   - You should focus on the CFG of the code, and you should treat each function and class as a complete unit.

### Example Output

Example code:

def example(a, b):
    if a > b:
        return a
    else:
        return b

class TestClass:
    def method_one(self, x):
        if x > 0:
            return x
        else:
            return -x

for i in range(3):
    print(i)

CFG in dot:
```dot
digraph CFG {
  subgraph cluster_Main {
    label="example_script";
    Block_1 [label="#1\\ndef example(a, b):...\\nclass TestClass:..."];
    Block_1 -> Block_21;
    Block_21 [label="#21\\nfor i in range(3):"];
    Block_21 -> Block_22;
    Block_22 [label="#22\\nprint(i)"];
    Block_22 -> Block_21;
  }
  subgraph cluster_Main_func_(1, 'example') {
    label="example";
    Block_3 [label="#3\\nif a > b:"];
    Block_3 -> Block_4;
    Block_4 [label="#4\\nreturn a"];
    Block_3 -> Block_6;
    Block_6 [label="#6\\nreturn b"];
  }
  subgraph cluster_Main_class_TestClass {
    label="TestClass";
    Block_11 [label="#11\\ndef method_one(self, x):..."];
  }
  subgraph cluster_Main_class_TestClass_func_(11, 'method_one') {
    label="method_one";
    Block_13 [label="#13\\nif x > 0:"];
    Block_13 -> Block_14;
    Block_14 [label="#14\\nreturn x"];
    Block_13 -> Block_16;
    Block_16 [label="#16\\nreturn -x"];
  }
}
```

Your Input Code:

""" + code_example + """

Please generate the CFG in DOT format according to the above instructions, and then parse it into a nested data structure as the following format:
```json
{
  "name": "...",
  "type": "CFG",
  "blocks": [
    {
      "id": int,
      "label": "...",
      "successors": [...]
    }
  ],
  "functions": [
    {
      "name": "...",
      "type": "function",
      "blocks": [...]
    }
  ],
  "classes": [
    {
      "name": "...",
      "type": "class",
      "blocks": [...]
    }
  ]
}
```
"""

from llm import get_llm_answers

# 调用 LLM 接口
res = get_llm_answers(prompt, model_name="deepseek-chat", require_json=True)
print(json.loads(res))
# 提取 DOT 内容
# dot_content = extract_dot_from_raw_text(res)
# print("Extracted DOT Content:\n", dot_content)

# # 解析 DOT 为目标结构
# cfg_dict = parse_dot_to_target_structure(dot_content)

# # 打印解析后的字典
# print(json.dumps(cfg_dict, indent=2))

# # 保存为 JSON 文件
# with open("cfg_recursive.json", "w") as f:
#     json.dump(cfg_dict, f, indent=2)

# print("Parsed CFG saved to 'cfg_recursive.json'.")



Proccesing request
Response received
{'name': 'Main', 'type': 'CFG', 'blocks': [{'id': 1, 'label': '#1\nx = 1', 'successors': [2]}, {'id': 2, 'label': '#2\ndef foo(a):...\nclass Bar:...', 'successors': []}], 'functions': [{'name': 'foo', 'type': 'function', 'blocks': [{'id': 3, 'label': '#3\nif a > 0:', 'successors': [4, 6]}, {'id': 4, 'label': '#4\nprint("positive")', 'successors': []}, {'id': 6, 'label': '#6\nprint("not positive")', 'successors': []}]}], 'classes': [{'name': 'Bar', 'type': 'class', 'blocks': [{'id': 11, 'label': '#11\ndef methodA(self, y):...\ndef methodB(self, z):...', 'successors': []}], 'functions': [{'name': 'methodA', 'type': 'function', 'blocks': [{'id': 13, 'label': '#13\nreturn y + 123', 'successors': []}]}, {'name': 'methodB', 'type': 'function', 'blocks': [{'id': 16, 'label': '#16\npass', 'successors': []}]}]}]}


In [12]:
from typing import Dict, List


def merge_blocks(cfg_dict: Dict) -> Dict:
    """递归合并CFG中具有唯一后继的节点"""
    
    def merge_block_list(blocks: List[Dict]) -> List[Dict]:
        # 如果blocks为空直接返回
        if not blocks:
            return blocks
            
        merged = []
        i = 0
        while i < len(blocks):
            curr = blocks[i]
            
            # 如果当前节点只有一个后继
            if len(curr.get("successors", [])) == 1:
                next_id = curr["successors"][0]
                # 找到后继节点
                next_block = next((b for b in blocks if b["id"] == next_id), None)
                
                if next_block:
                    # 合并label
                    curr["label"] = curr["label"] + "\n" + next_block["label"]
                    # 继承后继节点的后继
                    curr["successors"] = next_block.get("successors", [])
                    # 跳过后继节点
                    i += 2
                    merged.append(curr)
                    continue
            
            merged.append(curr)
            i += 1
            
        return merged

    # 递归处理主体blocks
    cfg_dict["blocks"] = merge_block_list(cfg_dict["blocks"])
    
    # 递归处理functions中的blocks
    for func in cfg_dict.get("functions", []):
        func["blocks"] = merge_block_list(func["blocks"])
        
    # 递归处理classes中的blocks
    for cls in cfg_dict.get("classes", []):
        cls["blocks"] = merge_block_list(cls["blocks"])
        # 处理类中的方法
        for method in cls.get("functions", []):
            method["blocks"] = merge_block_list(method["blocks"])
            
    return cfg_dict

# 解析JSON响应
cfg_dict = json.loads(res)

# 合并blocks
merged_cfg = merge_blocks(cfg_dict)


with open("cfg_recursive.json", "w") as f:
    json.dump(merged_cfg, f, indent=2)


## Refine Generate

In [9]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import json
from typing import List, Dict, Any, Optional
from dataclasses import dataclass, field
import logging

# 配置日志
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s'
)

#############################################
#   你的 LLM 接口 (需要你自己实现)
#############################################
from llm import get_llm_answers

#############################################
#   数据结构：Link, Block, CFG
#############################################

@dataclass
class Link:
    """控制流边：表示两个基本块之间的跳转关系"""
    source: str  # Block ID
    target: str  # Block ID
    condition: Optional[str] = None  # 跳转条件，使用字符串存储以便JSON序列化

    def to_dict(self) -> dict:
        return {
            'source': self.source,
            'target': self.target,
            'condition': self.condition
        }

@dataclass
class Block:
    """基本块：包含一系列顺序执行的语句"""
    id: str
    statements: List[str] = field(default_factory=list)  # 存储语句的字符串表示
    exits: List[Link] = field(default_factory=list)
    predecessors: List[str] = field(default_factory=list)  # Block IDs

    def to_dict(self) -> dict:
        return {
            'id': self.id,
            'statements': self.statements,
            'exits': [lk.to_dict() for lk in self.exits],
            'predecessors': self.predecessors
        }

@dataclass
class CFG:
    """控制流图：表示一个函数或模块的控制流"""
    name: str  # 函数/类/模块名称
    blocks: Dict[str, Block] = field(default_factory=dict)  # Block ID -> Block
    entry_block: Optional[str] = None  # Entry Block ID
    exit_blocks: List[str] = field(default_factory=list)  # Exit Block IDs

    functioncfgs: Dict[str, 'CFG'] = field(default_factory=dict)  
    class_cfgs: Dict[str, 'CFG'] = field(default_factory=dict)

    def to_dict(self) -> dict:
        return {
            'name': self.name,
            'blocks': {bid: blk.to_dict() for bid, blk in self.blocks.items()},
            'entry_block': self.entry_block,
            'exit_blocks': self.exit_blocks,
            'functioncfgs': {fn: subcfg.to_dict() for fn, subcfg in self.functioncfgs.items()},
            'class_cfgs': {cn: subcfg.to_dict() for cn, subcfg in self.class_cfgs.items()}
        }

    def add_block(self, block: Block):
        logging.debug(f"Adding block {block.id} to CFG {self.name}")
        self.blocks[block.id] = block

    def add_link(self, link: Link):
        """在 blocks 中找到 link.source, 给它加 exits，并更新 link.target 的 predecessors。"""
        if link.source in self.blocks and link.target in self.blocks:
            logging.debug(f"Adding link from {link.source} to {link.target} in CFG {self.name}")
            self.blocks[link.source].exits.append(link)
            if link.source not in self.blocks[link.target].predecessors:
                self.blocks[link.target].predecessors.append(link.source)

#############################################
#    Step A: 获取最外层块(保留 children)
#############################################

def stepA_get_hierarchical_blocks(whole_code: str) -> List[Dict[str, Any]]:
    """
    使用LLM，对 'whole_code' 做解析，生成**顶层**的块列表:
    每个块: { "decl_name":..., "start_line":..., "end_line":..., "code":"...", "children": [...] }
    不递归解析到 children. 保留 children 字段以便后续递归解析。
    """
    logging.info("Starting Step A: Hierarchical block analysis")
    prompt = """
You are a code analysis assistant. 
We want to split the following code into top-level blocks only. 
If there's a top-level class or function, treat its entire body as part of that single block's 'code', 
but include them in 'children' for further analysis.

Return strictly a JSON array, each element:
- decl_name (e.g. "GlobalBlock", "def foo", "class Bar")
- start_line
- end_line
- code (the lines of code that belong to this block)
- children (if any, an array of similar block objects)

No extra text.

Your output should be a json array, each element is a json object following the format:
[{
  "decl_name": "...",
  "start_line": int,
  "end_line": int,
  "code": "...",
  "children": [{...}, {...}]
},{
  "decl_name": "...",
  "start_line": int,
  "end_line": int,
  "code": "..."
}]
    
Code:
""" + whole_code
    raw_str = get_llm_answers(prompt, model_name="gpt-4o", require_json=True)
    blocks = json.loads(raw_str)

    if not isinstance(blocks, list):
        blocks = [blocks]

    logging.info(f"Step A completed: Found {len(blocks)} top-level blocks")
    return blocks

#############################################
#    Step B: 基本块划分
#############################################

def stepB_basic_block_segmentation(code_str: str) -> List[Dict[str, Any]]:
    """
    使用LLM，对 code_str 做"基本块"划分; 
    输出 { "basic_blocks": [ {start_line, end_line, code}, ... ] }
    并确保class和function作为整体块处理。
    """
    logging.info("Starting Step B: Basic block segmentation")
    prompt = """
You are a code analysis assistant.
Break the following code into basic blocks (straight-line segments). 
Ensure that class and function declarations along with their entire bodies are treated as single blocks and are not split into smaller blocks.
Return strictly JSON of the form:
{
  "basic_blocks": [
    {
      "start_line": int,
      "end_line": int,
      "code": "..."
    },
    ...
  ]
}
Code:
""" + code_str
    raw_json = get_llm_answers(prompt, model_name="deepseek-chat", require_json=True)
    parsed = json.loads(raw_json)
    logging.info(f"Step B completed: Found {len(parsed['basic_blocks'])} basic blocks")
    return parsed["basic_blocks"]

#############################################
#    Step C: 控制流分析
#############################################

def stepC_control_flow_analysis(basic_blocks: List[Dict[str, Any]], code_str: str) -> List[Dict[str,Any]]:
    """
    让 LLM 分析 basic_blocks 间的跳转. 返回:
    [
      {
        "block_id": 1,
        "successors": [2, 3],
        "conditions": ["", "x>0"]
      },
      ...
    ]
    """
    logging.info("Starting Step C: Control flow analysis")
    blocks_json = json.dumps(basic_blocks, indent=2, ensure_ascii=False)
    prompt = """
We have these basic blocks and the code. 
Please treat each block as a single entity, even if it contains multiple internal basic blocks.
Please produce the control flow edges in strictly JSON form, e.g.:
{
  "control_flow": [
    {
      "block_id": 1,
      "successors": [2],
      "conditions": [""]
    },
    ...
  ]
}


Code:
""" + code_str + """

Your output should obey the block I get from stepA. That's to say, you should not split the block into smaller blocks.
Blocks:
""" + blocks_json

    raw_cf = get_llm_answers(prompt, model_name="deepseek-chat", require_json=True)
    parsed = json.loads(raw_cf)
    logging.info(f"Step C completed: Found {len(parsed['control_flow'])} control flow edges")
    return parsed["control_flow"]

#############################################
#   构建CFG (支持递归解析嵌套的类和函数)
#############################################

def build_cfg_for_top_block(blk: Dict[str,Any], idx: int, processed_blocks: set) -> CFG:
    """
    根据 blk 的类型构建 CFG，同时避免重复处理同一个块。
    """
    name = blk.get("decl_name", f"Block{idx}")
    if name in processed_blocks:
        logging.warning(f"Block {name} 已经被处理，跳过以避免循环。")
        return CFG(name=name)  # 返回一个空的 CFG 或根据需求处理

    processed_blocks.add(name)
    logging.info(f"Building CFG for block: {name}")
    code_str = blk.get("code","")
    top_cfg = CFG(name=name)

    # 判断块的类型
    decl_name = name.strip()
    if decl_name.lower().startswith("globalblock"):
        logging.info(f"Processing global block {name}")
        # 对其 code 做 basic block + control flow
        basic_blocks = stepB_basic_block_segmentation(code_str)
        # 生成Block
        for i, bb in enumerate(basic_blocks, 1):
            block_id = f"{name}_block{i}"
            lines = bb.get("code","").split('\n')
            b = Block(id=block_id, statements=lines)
            top_cfg.add_block(b)
            if i == 1:
                top_cfg.entry_block = block_id
        # 做 stepC
        cflow = stepC_control_flow_analysis(basic_blocks, code_str)
        for item in cflow:
            from_id = f"{name}_block{item['block_id']}"
            succs = item["successors"]
            conds = item.get("conditions", [""]*len(succs))
            for si, sid in enumerate(succs):
                to_id = f"{name}_block{sid}"
                cond = conds[si] if si < len(conds) else ""
                lk = Link(source=from_id, target=to_id, condition=cond)
                top_cfg.add_link(lk)
        # exit blocks => 没有后继
        for bid, block_obj in top_cfg.blocks.items():
            if not block_obj.exits:
                top_cfg.exit_blocks.append(bid)
    elif decl_name.lower().startswith("def "):
        # 函数 => 解析其内部代码
        func_name = decl_name[4:].strip().split('(')[0]  # 提取函数名
        logging.info(f"Processing function: {func_name}")
        func_cfg = parse_code_to_cfg(code_str, filename=func_name, processed_blocks=processed_blocks)
        top_cfg.functioncfgs[func_name] = func_cfg
    elif decl_name.lower().startswith("class "):
        # 类 => 解析其内部代码
        class_name = decl_name[6:].strip().split('(')[0]  # 提取类名
        logging.info(f"Processing class: {class_name}")
        class_cfg = parse_code_to_cfg(code_str, filename=class_name, processed_blocks=processed_blocks)
        top_cfg.class_cfgs[class_name] = class_cfg
    else:
        # 其他类型，视为单条语句
        logging.info(f"Processing single statement block: {name}")
        block_id = f"{name}_single"
        lines = code_str.split('\n')
        b = Block(id=block_id, statements=lines)
        top_cfg.add_block(b)
        top_cfg.entry_block = block_id
        top_cfg.exit_blocks.append(block_id)

    # 处理 children
    children = blk.get("children", [])
    for child in children:
        logging.info(f"Processing child block in {name}")
        child_cfg = build_cfg_for_top_block(child, idx=idx, processed_blocks=processed_blocks)
        decl_child_name = child.get("decl_name", f"{name}_child{idx}")
        if decl_child_name.lower().startswith("def "):
            func_name = decl_child_name[4:].strip().split('(')[0]
            top_cfg.functioncfgs[func_name] = child_cfg
        elif decl_child_name.lower().startswith("class "):
            class_name = decl_child_name[6:].strip().split('(')[0]
            top_cfg.class_cfgs[class_name] = child_cfg
        else:
            # 其他类型，直接添加
            for block_id, block in child_cfg.blocks.items():
                top_cfg.add_block(block)
                if not top_cfg.entry_block:
                    top_cfg.entry_block = block_id
            top_cfg.exit_blocks.extend(child_cfg.exit_blocks)
            top_cfg.functioncfgs.update(child_cfg.functioncfgs)
            top_cfg.class_cfgs.update(child_cfg.class_cfgs)

    logging.info(f"Completed building CFG for {name}")
    return top_cfg

def parse_code_to_cfg(whole_code: str, filename: str="<unknown>", processed_blocks: set = None) -> CFG:
    """
    解析代码为 CFG，同时传递 processed_blocks 集合以避免重复处理。
    """
    if processed_blocks is None:
        processed_blocks = set()
    logging.info(f"Starting to parse code to CFG for {filename}")
    # 全局 CFG
    global_cfg = CFG(name=f"GlobalCFG({filename})")

    top_blocks = stepA_get_hierarchical_blocks(whole_code)

    # 依次处理每个顶层块
    for i, blk in enumerate(top_blocks, 1):
        cfg_i = build_cfg_for_top_block(blk, idx=i, processed_blocks=processed_blocks)
        # 依照 decl_name 判断是函数还是类
        decl_name = blk.get("decl_name", f"TopBlock{i}")
        if decl_name.lower().startswith("def "):
            func_name = decl_name[4:].strip().split('(')[0]
            global_cfg.functioncfgs[func_name] = cfg_i.functioncfgs.get(func_name, cfg_i)
        elif decl_name.lower().startswith("class "):
            class_name = decl_name[6:].strip().split('(')[0]
            global_cfg.class_cfgs[class_name] = cfg_i.class_cfgs.get(class_name, cfg_i)
        elif decl_name.lower().startswith("globalblock"):
            # 直接添加到 blocks
            for block_id, block in cfg_i.blocks.items():
                global_cfg.add_block(block)
                if not global_cfg.entry_block:
                    global_cfg.entry_block = block_id
            global_cfg.exit_blocks.extend(cfg_i.exit_blocks)
            # 合并 functioncfgs and class_cfgs if any
            global_cfg.functioncfgs.update(cfg_i.functioncfgs)
            global_cfg.class_cfgs.update(cfg_i.class_cfgs)
        else:
            # 其他类型，直接添加
            for block_id, block in cfg_i.blocks.items():
                global_cfg.add_block(block)
                if not global_cfg.entry_block:
                    global_cfg.entry_block = block_id
            global_cfg.exit_blocks.extend(cfg_i.exit_blocks)
            global_cfg.functioncfgs.update(cfg_i.functioncfgs)
            global_cfg.class_cfgs.update(cfg_i.class_cfgs)

    logging.info(f"Completed parsing code to CFG for {filename}")
    return global_cfg

#############################################
#  DEMO
#############################################

def main():
    code_example = """
x = 1

def foo(a):
    if a > 0:
        print("positive")
    else:
        print("not positive")

class Bar:
    def methodA(self, y):
        return y + 123
    def methodB(self, z):
        pass
"""
    print("=== DEMO parse_code_to_cfg ===")
    # 记得先实现 get_llm_answers
    final_cfg = parse_code_to_cfg(code_example, filename="demo.py")

    print("Global CFG name =", final_cfg.name)
    print("Top-level sub-CFGs:", list(final_cfg.functioncfgs.keys()) + list(final_cfg.class_cfgs.keys()))
    for sub_name, sub_cfg in final_cfg.functioncfgs.items():
        print(f"  >> Function CFG {sub_name}: ")
        print("     entry =", sub_cfg.entry_block)
        print("     exit =", sub_cfg.exit_blocks)
        print("     blocks =", list(sub_cfg.blocks.keys()))
        # 也可以 to_dict() 后再 json.dumps() 看结构

    for class_name, class_cfg in final_cfg.class_cfgs.items():
        print(f"  >> Class CFG {class_name}: ")
        print("     entry =", class_cfg.entry_block)
        print("     exit =", class_cfg.exit_blocks)
        print("     blocks =", list(class_cfg.blocks.keys()))
        # 也可以 to_dict() 后再 json.dumps() 看结构

    # 如果需要，可以打印整个 CFG 结构
    print(json.dumps(final_cfg.to_dict(), indent=2, ensure_ascii=False))

if __name__ == "__main__":
    main()

2025-01-10 16:27:13,297 - INFO - Starting to parse code to CFG for demo.py
2025-01-10 16:27:13,299 - INFO - Starting Step A: Hierarchical block analysis


=== DEMO parse_code_to_cfg ===
Proccesing request


2025-01-10 16:27:20,232 - INFO - HTTP Request: POST https://xiaoai.plus/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:27:20,242 - INFO - Step A completed: Found 1 top-level blocks
2025-01-10 16:27:20,244 - INFO - Building CFG for block: GlobalBlock
2025-01-10 16:27:20,246 - INFO - Processing global block GlobalBlock
2025-01-10 16:27:20,247 - INFO - Starting Step B: Basic block segmentation


Proccesing request


2025-01-10 16:27:20,499 - INFO - HTTP Request: POST https://api.deepseek.com/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:27:24,944 - INFO - Step B completed: Found 3 basic blocks


...

Response received


2025-01-10 16:28:41,755 - INFO - Processing child block in GlobalBlock
2025-01-10 16:28:41,757 - INFO - Building CFG for block: def foo
2025-01-10 16:28:41,760 - INFO - Processing function: foo
2025-01-10 16:28:41,762 - INFO - Starting to parse code to CFG for foo
2025-01-10 16:28:41,763 - INFO - Starting Step A: Hierarchical block analysis


Proccesing request


2025-01-10 16:28:46,048 - INFO - HTTP Request: POST https://xiaoai.plus/v1/chat/completions "HTTP/1.1 200 OK"
2025-01-10 16:28:46,054 - INFO - Step A completed: Found 1 top-level blocks
2025-01-10 16:28:46,056 - WARNING - Block def foo 已经被处理，跳过以避免循环。
2025-01-10 16:28:46,058 - INFO - Completed parsing code to CFG for foo
2025-01-10 16:28:46,059 - INFO - Completed building CFG for def foo
2025-01-10 16:28:46,061 - INFO - Processing child block in GlobalBlock
2025-01-10 16:28:46,062 - INFO - Building CFG for block: class Bar
2025-01-10 16:28:46,065 - INFO - Processing class: Bar
2025-01-10 16:28:46,067 - INFO - Starting to parse code to CFG for Bar
2025-01-10 16:28:46,068 - INFO - Starting Step A: Hierarchical block analysis


Proccesing request
